In [3]:
import pandas as pd
from dateutil.relativedelta import relativedelta

In [4]:
datos_basicos=pd.read_csv("../Bases limpias/DatosBasicosL_20_jun_GM.csv")
habitos=pd.read_csv("../Bases limpias/HabitosL_20_jun_GM.csv")
RAM=pd.read_excel("../Bases limpias/Farmaco_RAM_limpia_18_jun_AM.xlsx")
Calidad_Vida=pd.read_excel("../Bases limpias/Calidad de vidaNew_20_jun_AG.xlsx")#
ACT=pd.read_csv("../Bases limpias/ACT_consolidada_26_junio_Ariel.csv")    


In [5]:
datos_basicos.rename(columns={"ID":"ID_PACIENTE"},inplace=True)
habitos.rename(columns={"DS_IDENTIFICACION":"ID_PACIENTE"},inplace=True)
habitos.FE_REGISTRO=pd.to_datetime(habitos.FE_REGISTRO)
RAM.sort_values("FECHA_INICIO_REACCION")
datos_basicos["BIRTHDATE"]=pd.to_datetime(datos_basicos["BIRTHDATE"])

Calidad_Vida=Calidad_Vida.loc[:,"ID_PACIENTE":"0_100"].pivot_table(index=["ID_PACIENTE","FE_ALTA"],columns="DIMENSIONES",values="0_100").reset_index().sort_values("FE_ALTA")


In [6]:
ACT=ACT.loc[:,['FE_RESULTADO', 'ID', 'NM_PUNTAJE']].sort_values('FE_RESULTADO')
ACT.rename(columns={ 'ID':"ID_PACIENTE", 'NM_PUNTAJE':'ACT_NM_PUNTAJE'},inplace=True)



In [1]:
def try_year(a): 
    try: 
        return relativedelta(a.FECHA_EMISION,a.BIRTHDATE).years 
    except: 
        return pd.NaT


def juntar_con_limpias(filename):
    _base_entrenamiento_=pd.read_excel(filename)
    print(_base_entrenamiento_.shape)
    _base_entrenamiento_.FECHA_EMISION=pd.to_datetime(_base_entrenamiento_.FECHA_EMISION)
    _db_=datos_basicos.loc[:,['ID_PACIENTE', 'SEX','DEPARTMENT', 'CITY', 'STRATUM', 'SOCIOECONOMIC_LEVEL','AFFILIATION_TYPE', 'BIRTHDATE', 'Education','MaritalStatus', 'LifeCyle', 'WorkingStatus', 'Zone', 'Ocupation']]
    _base_entrenamiento_=_base_entrenamiento_.merge(_db_,how="left")
    _base_entrenamiento_["Edad"]=_base_entrenamiento_.apply(try_year,axis=1)
    
    _habitos_=habitos.loc[habitos.Types.isin(["Pets","Alcohol","Tobacco","Exercise","Mood"]),["ID_PACIENTE","FE_REGISTRO","Types","Habits"]]
    #return _base_entrenamiento_.FECHA_EMISION.isna().sum()
    _habitos_=_habitos_.pivot_table(index=["ID_PACIENTE","FE_REGISTRO"],columns="Types",values="Habits",aggfunc='first').reset_index()
    _habitos_=_habitos_.sort_values("FE_REGISTRO")
    _base_entrenamiento_=_base_entrenamiento_.sort_values("FECHA_EMISION")
    _base_entrenamiento_=pd.merge_asof(_base_entrenamiento_,_habitos_,by="ID_PACIENTE",right_on="FE_REGISTRO",left_on="FECHA_EMISION")
    
    _RAM_=RAM[~RAM["FECHA_INICIO_REACCION"].isnull()]
    _RAM_=_RAM_.sort_values("FECHA_INICIO_REACCION").loc[:,["ID_PACIENTE",'MEDICAMENTO_SOSPECHOSO', 'FECHA_INICIO_REACCION']]
    _base_entrenamiento_=pd.merge_asof(_base_entrenamiento_,_RAM_,by="ID_PACIENTE",right_on="FECHA_INICIO_REACCION",left_on="FECHA_EMISION",direction="forward")
    _base_entrenamiento_["Ram"]= (_base_entrenamiento_["FechaSiguienteRecla"]>_base_entrenamiento_["FECHA_INICIO_REACCION"])
    _base_entrenamiento_["FECHA_INICIO_REACCION"][~_base_entrenamiento_["Ram"]]=pd.NaT
    _base_entrenamiento_["MEDICAMENTO_SOSPECHOSO"][~_base_entrenamiento_["Ram"]]=pd.NaT
    
    if (_base_entrenamiento_.columns=="Categoria").any():
        b="Categoria"
    else:
        b="NOM_GENERICO"
    _base_entrenamiento_["RamEspecifica"]=_base_entrenamiento_["MEDICAMENTO_SOSPECHOSO"].str.contains(_base_entrenamiento_[b][0])
    

    _base_entrenamiento_=pd.merge_asof(_base_entrenamiento_,Calidad_Vida,by="ID_PACIENTE",right_on="FE_ALTA",left_on="FECHA_EMISION")

    _base_entrenamiento_=pd.merge_asof(_base_entrenamiento_,ACT,by="ID_PACIENTE",right_on="FE_ALTA",left_on="FECHA_EMISION")

    print(_base_entrenamiento_.shape)
    _base_entrenamiento_.to_csv(filename.rsplit(".",1)[0]+"_trabajo.csv")

    return _base_entrenamiento_#_base_entrenamiento_


In [2]:
a=juntar_con_limpias("Train_BUDESONIDA.xlsx")
a

NameError: name 'pd' is not defined

In [ ]:
a.loc[213,:]